In [1]:
pip install gradio groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
"""
Healthcare Chatbot using Hugging Face and IBM Granite
No API keys required - runs entirely in Google Colab
"""

# Install required packages
!pip install -q transformers torch accelerate bitsandbytes gradio

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr
from typing import List, Tuple
import warnings
warnings.filterwarnings('ignore')

class HealthcareChatbot:
    def __init__(self):
        """Initialize the healthcare chatbot with IBM Granite model"""
        self.model_name = "ibm-granite/granite-3.0-2b-instruct"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.conversation_history = []

        print(f"Loading model on {self.device}...")
        self.load_model()
        print("Model loaded successfully!")

    def load_model(self):
        """Load the IBM Granite model and tokenizer"""
        try:
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )

            # Load model with optimizations for Colab
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto",
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )

            # Create pipeline
            self.pipe = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )

        except Exception as e:
            print(f"Error loading model: {e}")
            raise

    def create_healthcare_prompt(self, user_message: str, history: List) -> str:
        """Create a healthcare-focused prompt with context"""

        system_prompt = """You are a helpful and empathetic healthcare assistant. Your role is to:
- Provide general health information and wellness advice
- Answer questions about common medical conditions and symptoms
- Offer guidance on healthy lifestyle choices
- Suggest when to seek professional medical care
- Be supportive and understanding

IMPORTANT DISCLAIMERS:
- You are NOT a replacement for professional medical advice
- Always recommend consulting healthcare professionals for diagnosis
- Never provide specific medication dosages or treatment plans
- In emergencies, always advise calling emergency services

Be clear, concise, and compassionate in your responses."""

        # Build conversation context
        conversation = f"{system_prompt}\n\n"

        # Add history
        for user_msg, bot_msg in history[-3:]:  # Last 3 exchanges for context
            conversation += f"Patient: {user_msg}\nAssistant: {bot_msg}\n\n"

        # Add current message
        conversation += f"Patient: {user_message}\nAssistant:"

        return conversation

    def generate_response(self, message: str, history: List[Tuple[str, str]]) -> str:
        """Generate response to user message"""

        if not message.strip():
            return "Please ask me a health-related question."

        # Create prompt
        prompt = self.create_healthcare_prompt(message, history)

        try:
            # Generate response
            response = self.pipe(
                prompt,
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.15
            )[0]['generated_text']

            # Extract only the assistant's response
            assistant_response = response.split("Assistant:")[-1].strip()

            # Clean up response
            assistant_response = assistant_response.split("Patient:")[0].strip()

            # Add medical disclaimer if not present
            if "medical professional" not in assistant_response.lower() and len(assistant_response) > 100:
                assistant_response += "\n\n⚕️ *Remember: This is general information. Always consult a healthcare professional for personalized medical advice.*"

            return assistant_response

        except Exception as e:
            return f"I apologize, but I encountered an error. Please try rephrasing your question. Error: {str(e)}"

    def create_interface(self):
        """Create Gradio interface"""

        # Custom CSS for healthcare theme
        custom_css = """
        .gradio-container {
            font-family: 'Arial', sans-serif;
        }
        .health-header {
            text-align: center;
            color: #2c3e50;
            padding: 20px;
        }
        """

        with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
            gr.Markdown(
                """
                # 🏥 Healthcare Assistant Chatbot
                ### Powered by IBM Granite & Hugging Face

                Ask me about:
                - Common health symptoms and conditions
                - Wellness and prevention tips
                - Nutrition and exercise advice
                - Mental health support
                - When to see a doctor

                **⚠️ Important:** This chatbot provides general information only and is not a substitute for professional medical advice.
                """
            )

            chatbot = gr.Chatbot(
                height=500,
                bubble_full_width=False,
                avatar_images=(None, "🏥")
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your health question here...",
                    show_label=False,
                    scale=4
                )
                submit = gr.Button("Send", scale=1, variant="primary")

            with gr.Row():
                clear = gr.Button("Clear Conversation")

            gr.Markdown(
                """
                ### 📋 Sample Questions:
                - "What are the symptoms of dehydration?"
                - "How can I improve my sleep quality?"
                - "What should I know about managing stress?"
                - "When should I see a doctor for a headache?"
                """
            )

            # Emergency information
            gr.Markdown(
                """
                ---
                ### 🚨 Emergency Numbers:
                - **USA:** 911
                - **UK:** 999 or 111 (non-emergency)
                - **EU:** 112
                - **India:** 102 or 108

                **If you're experiencing a medical emergency, call emergency services immediately!**
                """
            )

            # Event handlers
            def respond(message, chat_history):
                bot_message = self.generate_response(message, chat_history)
                chat_history.append((message, bot_message))
                return "", chat_history

            msg.submit(respond, [msg, chatbot], [msg, chatbot])
            submit.click(respond, [msg, chatbot], [msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)

        return demo

# Initialize and launch the chatbot
print("🏥 Initializing Healthcare Chatbot...")
print("=" * 50)

try:
    bot = HealthcareChatbot()
    demo = bot.create_interface()

    print("\n✅ Chatbot ready!")
    print("=" * 50)

    # Launch with public link for Colab
    demo.launch(
        share=True,  # Creates public link
        debug=True,
        show_error=True
    )

except Exception as e:
    print(f"\n❌ Error initializing chatbot: {e}")
    print("\nTroubleshooting tips:")
    print("1. Make sure you have enough RAM (use GPU runtime)")
    print("2. Restart runtime and try again")
    print("3. Check your internet connection")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.5 MB/s eta 0:00:00


🏥 Initializing Healthcare Chatbot...
Loading model on cpu...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Model loaded successfully!

✅ Chatbot ready!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bf28a8a21aef9a46cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
